<a href="https://colab.research.google.com/github/sizhky/AutoTrain/blob/master/nbs/classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup Data and Libraries

In [1]:
%%capture
# Setup imports
try:
    import timm
except:
    !pip install -U torch-snippets typer imgaug timm fastai

In [2]:
# Setup Data
%load_ext autoreload
%autoreload 2

from torch_snippets import *
from fastai.vision.all import *

# If running locally from AutoTrain/nbs
if not os.path.exists('/content') and P().resolve().stem != 'AutoTrain':
    %cd ..

# If running from colab
if os.path.exists('/content') and not os.path.exists('/content/AutoTrain'):
    !git clone https://github.com/sizhky/AutoTrain

# change present working directory
if not P().resolve().stem == 'AutoTrain':
    %cd AutoTrain

# !git pull origin enhancement/fastai-classification
# !git checkout enhancement/fastai-classification

if not os.path.exists('imagenette2-160'):
    o = untar_data(URLs.IMAGENETTE_160)
    !cp -r {o} ./


2021-09-12 10:54:00.601 | WARNING  | torch_snippets:<module>:13 - sklearn is not found. Skipping relevant imports from submodule `sklegos`
Exception: No module named 'sklego'


/content/AutoTrain


## Verify Config

In [6]:
!cat configs/classification_imagenette.ini

[project]
owner: sizhky
name: imagenette
location: ${project.name}
model_directory: ${project.location}/models
type: classification
num_classes: 10

[label_studio]
base_data_dir: /tmp/data/

[label_studio_ml]
app_name: ${project.name}/my_ml_model
script_path: auto_train_classification/script_classification.py

[online_training]
save_dir: ${project.model_directory}/model.pt

[architecture]
image_size: 224
[architecture.backbone]
model: efficientnet_b0
# everytime you change the model, vector size 
# must match the model's backbone output.
vector_size: 1280
[architecture.head]
@head: custom_head
head_input_size: ${architecture.backbone.vector_size}
num_classes: ${project.num_classes}

[training]
dir: imagenette2-160
train_ratio: 0.8
[training.preprocess]
@preprocess_function: my_preprocess
image_size: ${architecture.image_size}

[training.data]
train_dir: ${training.dir}/train
validation_dir: ${training.dir}/validation

[training.scheme]
epochs: 2
freeze_epochs: 2
# initial_checkpoint: $

In [4]:
os.environ['CONFIG'] = 'configs/classification_imagenette.ini'
!python auto_train_classification/train.py

2021-09-12 10:54:08.444 | WARNING  | torch_snippets:<module>:13 - sklearn is not found. Skipping relevant imports from submodule `sklegos`
Exception: No module named 'sklego'
2021-09-12 10:54:12.406 | INFO     | __main__:train_model:35 - Training from scratch!
2021-09-12 10:54:44.110 | INFO     | __main__:find_best_learning_rate:22 - LR Plot is saved at imagenette/find_lr_plot.png
2021-09-12 10:54:44.111 | INFO     | __main__:train_model:40 - Using lr: 0.00013182566908653826
(2, 0.00013182566908653826, 2)
['Epoch, Train Loss, Validation Loss, Validation Accuracy, Time']
[0, 1.8888294696807861, 1.5228769779205322, 0.6402547955513, '00:51']
[1, 0.9272148609161377, 0.628566563129425, 0.8568152785301208, '00:50']
[0, 0.6768947243690491, 0.5767525434494019, 0.8654776811599731, '00:59']
[1, 0.6188012957572937, 0.5099591612815857, 0.8672611713409424, '00:59']
2021-09-12 10:58:24.313 | INFO     | __main__:train_model:55 - Saved weights of size ~18 MB to imagenette/models/trained-model-v1/mymod

---
## Validate

In [5]:
from auto_train_classification.infer import infer
infer('imagenette2-160/val/n01440764/')

2021-09-12 10:58:32.598 | INFO     | auto_train_classification.infer:<module>:6 - Loaded weights from imagenette/models/trained-model-v1/mymodel.pt to given model


[(» imagenette2-160/val/n01440764/n01440764_2290.JPEG,
  'n01440764',
  tensor(0.6419)),
 (» imagenette2-160/val/n01440764/n01440764_12111.JPEG,
  'n01440764',
  tensor(0.9974)),
 (» imagenette2-160/val/n01440764/n01440764_7571.JPEG,
  'n03394916',
  tensor(0.2307)),
 (» imagenette2-160/val/n01440764/n01440764_31170.JPEG,
  'n01440764',
  tensor(0.9769))]